In [ ]:
# Import functions
from pyspark.sql.functions import col, current_timestamp,input_file_name

# Define variables used in code below
file_path = "/FileStore/stream/events/"
username = spark.sql("SELECT regexp_replace(current_user(), '[^a-zA-Z0-9]', '_')").first()[0]
table_name = f"{username}_etl_quickstart"
checkpoint_path = f"/tmp/{username}/_checkpoint/etl_quickstart"

# Clear out data from previous demo execution
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
dbutils.fs.rm(checkpoint_path, True)

In [ ]:
dbutils.fs.rm(f"dbfs:/user/hive/warehouse/{username}_etl_quickstart", True)

In [ ]:
%fs ls /databricks-datasets/structured-streaming/events

In [ ]:
dbutils.fs.cp("/databricks-datasets/structured-streaming/events/file-4.json","/FileStore/stream/events/file-4.json",True)

In [ ]:
display(dbutils.fs.ls("/FileStore/stream/events/"))

In [ ]:
spark.read.format("json").load("dbfs:/FileStore/stream/events/").count()

In [ ]:
schema_location = f"/tmp/{username}/schema/etl_quickstart"
(spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "json")
  .option("cloudFiles.schemaLocation", schema_location) 

  .load(file_path)

  .withColumn("source_file", input_file_name()) 
  .writeStream
  .option("checkpointLocation", checkpoint_path)

  .trigger(availableNow=True)
  .toTable(table_name))  

df = spark.sql(f"SELECT * FROM {table_name} LIMIT 10")
df.display()

In [ ]:
df = spark.sql(f"select count(*) from {table_name}")
df.display()

In [ ]:
display(dbutils.fs.ls(f"/tmp/{username}/_checkpoint/etl_quickstart/sources/0/metadata/"))

In [ ]:
display(dbutils.fs.ls(f"/tmp/{username}/_checkpoint/etl_quickstart/sources/0/metadata/"))